In [1]:
#pip uninstall -y enum34

In [2]:
#pip install opencv-contrib-python

In [3]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
import xml.etree.ElementTree as ET
import numpy as np
import PIL.Image as Image

/opt/conda/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [5]:
cropped_truck_images_path = "Labeled_Truck_Dataset/"

In [6]:
def read_pascal(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []
    
    filename = root.find('filename').text
    filename= filename.split('.')[0]
    for boxes in root.iter('object'):     
        name = boxes.find("name").text     
        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            ymin = int(float(box.find("ymin").text))
            xmin = int(float(box.find("xmin").text))
            ymax = int(float(box.find("ymax").text))
            xmax = int(float(box.find("xmax").text))          

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append((list_with_single_boxes, name))
        
    return filename, list_with_all_boxes

In [7]:
def compute_iou(boxA, boxB):
# determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [8]:
Brand_path = 'Detected_Data/Brand/'
Plate_path = 'Detected_Data/Plate/'
Useless_path = 'Detected_Data/Useless/'

In [9]:
def detect_object(found_box_use, box, name, directory_path, org_file_name, object_count):
    iou = compute_iou(found_box_use, box) 
    if iou > 0.7:  
        print(iou)
        print('There is a '+ name)
        image_roi_use = cv2.resize(image_roi,(128,128))
        image_roi_use = cv2.cvtColor(image_roi_use, cv2.COLOR_BGR2RGB)
        image_roi_use = Image.fromarray(image_roi_use)
        file_path = directory_path+name+'/'+org_file_name+'_'+'_'+str(object_count)+'.jpg'
        image_roi_use.save(file_path)                                               
        return True 
    
    return iou

In [10]:
def save_useless(image_roi, Useless_path, org_file_name, useless_count):
    image_roi_use = cv2.resize(image_roi,(128,128))
    image_roi_use = cv2.cvtColor(image_roi_use, cv2.COLOR_BGR2RGB)
    image_roi_use = Image.fromarray(image_roi_use)
    file_path = Useless_path+org_file_name+'_useless_'+str(useless_count)+'.jpg'
    image_roi_use.save(file_path)            

In [11]:
count_image_detected = 0
for file in os.listdir(cropped_truck_images_path):
    if '.jpg' in file:
        org_file_name, box_list = read_pascal(cropped_truck_images_path+file.split('.')[0]+'.xml')
        
        plate_box = box_list[0]
        brand_box = box_list[1]
        
        if plate_box[1].lower() != "plate":
            temp = plate_box
            plate_box = brand_box
            brand_box = temp
            
            
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
        pic = cv2.imread(cropped_truck_images_path+file)
        pic_copy = pic.copy()
        
        ss.setBaseImage(pic)
        ss.switchToSelectiveSearchFast()
        results = ss.process()
        
        count_brand_image = 0
        count_plate_image = 0
        count_useless_image = 0
        count_boxes_one_img = 0
        
        for found_box in results:
            found_box_use = [found_box[0],found_box[1],found_box[0]+found_box[2],found_box[1]+found_box[3]]
            image_roi = pic_copy[found_box[1]:found_box[3]+found_box[1],found_box[0]:found_box[0]+found_box[2]]        
                        
            
            iou = 0
            if count_plate_image < 10:
                iou = detect_object(found_box_use, plate_box[0], plate_box[1], Plate_path, org_file_name, count_plate_image)
                if iou == True:
                    count_plate_image+=1
                    continue            
            
            ### BRAND Detection
            if count_brand_image < 10:
                iou = detect_object(found_box_use, brand_box[0], brand_box[1], Brand_path, org_file_name, count_brand_image) 
                if iou == True:
                    count_brand_image+=1
                    continue   
            
            if iou<0.3 and count_useless_image < 10:      
                save_useless(image_roi, Useless_path, org_file_name, count_useless_image)
                count_useless_image += 1            
            
            if count_boxes_one_img >= 1000 or (count_plate_image>=10 and count_brand_image>=10):
                break         
                
            count_boxes_one_img += 1

0.7141698215251487
There is a PLATE
0.7345154845154845
There is a PLATE
0.7471867966991748
There is a MAN
0.8383037829888063
There is a PLATE
0.7373965387509406
There is a MAN
0.7129380053908356
There is a PLATE
0.8369272237196765
There is a PLATE
0.8522727272727273
There is a PLATE
0.8460242587601078
There is a PLATE
0.8294295157350194
There is a PLATE
0.8505021520803443
There is a PLATE
0.8294295157350194
There is a PLATE
0.7499464782701777
There is a PLATE
0.7493389740877843
There is a PLATE
0.871849377467355
There is a SCANIA
0.8116758933064921
There is a MERCEDES
0.9017180663749866
There is a MERCEDES
0.8626977892492426
There is a MERCEDES
0.8910833575370316
There is a PLATE
0.7864406779661017
There is a PLATE
0.7524752475247525
There is a PLATE
0.7341772151898734
There is a PLATE
0.7429245283018868
There is a IVECO
0.711864406779661
There is a PLATE
0.891912320483749
There is a PLATE
0.7443609022556391
There is a PLATE
0.7492063492063492
There is a PLATE
0.7372881355932204
There 

0.7273449920508744
There is a PLATE
0.921793534932221
There is a DAF
0.7591240875912408
There is a DAF
0.7639109697933227
There is a PLATE
0.8033826638477801
There is a RENO
0.8846308995562727
There is a PLATE
0.7875647668393783
There is a RENO
0.9066253390158853
There is a PLATE
0.8786057692307693
There is a PLATE
0.7486889874949576
There is a PLATE
0.8888080072793448
There is a PLATE
0.7095435684647303
There is a RENO
0.73125
There is a PLATE
0.7764705882352941
There is a PLATE
0.8694329784922876
There is a DAF
0.715970515970516
There is a DAF
0.8692349430276722
There is a PLATE
0.8811939657486444
There is a SCANIA
0.8297728805936586
There is a PLATE
0.7428571428571429
There is a PLATE
0.7743789776226647
There is a PLATE
0.9148727984344422
There is a PLATE
0.8195364238410596
There is a PLATE
0.8047045509659688
There is a PLATE
0.7133362871067789
There is a PLATE
